In [1]:
import sqlite3
import requests
import string

In [2]:
#API root and auth
sale_id_api = 'https://logistics-api-dot-active-thunder-329100.rj.r.appspot.com'
chip_api = 'https://telecom-api-dot-active-thunder-329100.rj.r.appspot.com'
conversations_api = 'https://chats-api-dot-active-thunder-329100.rj.r.appspot.com'
header = {'authorization': 'teste'} #auth code

#HTTP Requests
#   Sale API
HTTP_request_sale_track = '/tracking'
HTTP_request_sale_zip = '/zip_code'
#   Chip Api
HTTP_request_chip = '/chip_status'
#   Conversation API
HTTP_request_conversation = '/conversations'
HTTP_request_conversation_info = '/conversation_info'

# SQL tools

In [3]:
#get for the first act conversation /!\ need to do it for all act convers
def get_sale_id_SQL(merchant_id): #for now initialize to first act conv
#     conversation=generate_act_conv_info
#     conversation = conversation[0] #TBDelt when implemented on asw_convers
    
#     merchant_id = conversation.get('merchant_id')
    
    #/!\ SQL Hardcoded Implement Cloud SQL on GCP
    conn = sqlite3.connect('/home/ecapi/code/e-capi/Chat_bot_Software_Engineer/Chat_bot_Software_Engineer/data/simulation-db')
    c = conn.cursor()
    query = """
        SELECT id_sale
        FROM sales
        WHERE sales.merchant_id = ?
    """ 
    c.execute(query, (merchant_id,))
    sale_id = c.fetchone()
    return {"id_sale": str(sale_id[0])} #gives a tupple, with a second null arg

In [4]:
#get for the first act conversation /!\ need to do it for all act convers
def get_receipt_info_SQL(merchant_id): #for now initialize to first act conv
#     conversation=generate_act_conv_info
#     conversation = conversation[0] #TBDelt when implemented on asw_convers
    
#     merchant_id = conversation.get('merchant_id')
    
    #/!\ SQL Hardcoded Implement Cloud SQL on GCP
    conn = sqlite3.connect('/home/ecapi/code/e-capi/Chat_bot_Software_Engineer/Chat_bot_Software_Engineer/data/simulation-db')
    c = conn.cursor()
    query = """
        SELECT status, description, value
        FROM receipt
        WHERE receipt.merchant_id = ?
    """ 
    c.execute(query, (merchant_id,))
    receipt_info = c.fetchone()
    
    #variables
    status = receipt_info[0]
    description = receipt_info[1]
    value = receipt_info[2]
    
    answ_dict = {'status':status, 'description': description, 'value': value }
    return answ_dict

In [5]:
#get for the first act conversation /!\ need to do it for all act convers
def get_chip_id_SQL(merchant_id): #for now initialize to first act conv
#     conversation=generate_act_conv_info
#     conversation = conversation[0] #TBDelt when implemented on asw_convers
    
#     merchant_id = conversation.get('merchant_id')
    
    #/!\ SQL Hardcoded Implement Cloud SQL on GCP
    conn = sqlite3.connect('/home/ecapi/code/e-capi/Chat_bot_Software_Engineer/Chat_bot_Software_Engineer/data/simulation-db')
    c = conn.cursor()
    query = """
        SELECT chip_id
        FROM sales
        WHERE sales.merchant_id = ?
    """ 
    c.execute(query, (merchant_id,))
    chip_id = c.fetchone()
    return {"chip_id": str(chip_id[0])} #gives a tupple, with a second null arg

In [6]:
#get for the first act conversation /!\ need to do it for all act convers
def get_transactions_info_SQL(merchant_id): #for now initialize to first act conv
#     conversation=generate_act_conv_info
#     conversation = conversation[0] #TBDelt when implemented on asw_convers
    
#     merchant_id = conversation.get('merchant_id')
    
    #/!\ SQL Hardcoded Implement Cloud SQL on GCP
    conn = sqlite3.connect('/home/ecapi/code/e-capi/Chat_bot_Software_Engineer/Chat_bot_Software_Engineer/data/simulation-db')
    c = conn.cursor()
    query = """
        SELECT transaction_id, created_at, value 
        FROM transactions
        WHERE transactions.merchant_id = ?
    """ 
    c.execute(query, (merchant_id,))
    transactions_info = c.fetchall()
    
    #variables
    
#     answ_dict = {'status':status, 'description': description, 'value': value }
    return transactions_info #returns a list of transaction_id, created_at, value

In [7]:
#get for the first act conversation /!\ need to do it for all act convers
def get_transactions_aggregate_SQL(merchant_id): #for now initialize to first act conv
#     conversation=generate_act_conv_info
#     conversation = conversation[0] #TBDelt when implemented on asw_convers
    
#     merchant_id = conversation.get('merchant_id')
    
    #/!\ SQL Hardcoded Implement Cloud SQL on GCP
    conn = sqlite3.connect('/home/ecapi/code/e-capi/Chat_bot_Software_Engineer/Chat_bot_Software_Engineer/data/simulation-db')
    c = conn.cursor()
    query = """
        SELECT COUNT(transaction_id) as total_transactions, SUM(value) as total_value
        FROM transactions
        WHERE transactions.merchant_id = ?
    """ 
    c.execute(query, (merchant_id,))
    transactions_aggregation = c.fetchone()
    
    #variables
    transactions_amount = transactions_aggregation[0]
    transactions_value = transactions_aggregation[1]
    
    
    answ_dict = {'transactions_amount':transactions_amount, 'transactions_value': transactions_value }
    return answ_dict

# Bot tools

In [8]:
def generate_sale_tracking_info(params_request):
    url = sale_id_api+HTTP_request_sale_track

    response_sale_track = requests.post(url, json=params_request, headers=header)
    return response_sale_track.json()

In [9]:
def generate_sale_zip_info(params_request):
    url = sale_id_api+HTTP_request_sale_zip 

    response_sale_zip = requests.post(url, json=params_request, headers=header)
    return response_sale_zip.json()
# def generate_sale_zip_info(params_request):
#     url = sale_id_api+HTTP_request_sale_zip

#     response_sale_zip = requests.post(url, json={'zip_code':params_request}, headers=header)
#     return response_sale_zip.json()

In [10]:
def generate_chip_info(params_request):
    url = chip_api+HTTP_request_chip

    response_chip = requests.post(url, json=params_request, headers=header)
    return response_chip.json()

In [11]:
#all convs info list
def generate_all_act_conv_info():
    url = conversations_api+HTTP_request_conversation
    response_conversation = requests.post(url, headers=header)
    json_response_conversation = response_conversation.json() #retrieve active conversations ID

    active_chat_list = [] #Create a list of active conversations info
    for values in json_response_conversation.values():
        for val in values:
            params_conver_info = {'conversation_id': str(val)}
            response_conversation = requests.post(conversations_api+HTTP_request_conversation_info, headers=header, json=params_conver_info)
            active_chat_list.append(response_conversation.json())

        return active_chat_list

In [12]:
generate_all_act_conv_info() 

[{'conversation_id': '754893',
  'messages': [{'message_id': '754893-0',
    'text': "hello, my money hasn't landed in my bank account yet. I want to know what happened",
    'created_at': '1640197668'}],
  'merchant_id': '558392',
  'subject': 'bank account receipt problem'},
 {'conversation_id': '754894',
  'messages': [{'message_id': '754894-0',
    'text': "my chip is not connected to the internet. I've already tested my wifi and it's ok, the problem is on the chip itself",
    'created_at': '1640197670'}],
  'merchant_id': '530364',
  'subject': 'connection problem'},
 {'conversation_id': '754895',
  'messages': [{'message_id': '754895-0',
    'text': "where is my credit card machine? it hasn't arrived yet, the delivery is already late",
    'created_at': '1640197673'}],
  'merchant_id': '384923',
  'subject': 'late delivery'},
 {'conversation_id': '754896',
  'messages': [{'message_id': '754896-0',
    'text': "Hi! I wanted to know where my machine is. I'm going on a vacation nex

In [13]:
#active conv list
def generate_act_convs_list():
    url = conversations_api+HTTP_request_conversation
    response_conversation = requests.post(url, headers=header)
    json_response_conversation = response_conversation.json() #retrieve active conversations ID
    
    active_convs_list = json_response_conversation.get('conversation_ids')
    return active_convs_list

In [14]:
generate_act_convs_list()

[754893, 754894, 754895, 754896, 754897]

In [15]:
def generate_act_conv_info(conversation_id):
    
    url = conversations_api+HTTP_request_conversation_info
    params = {"conversation_id": conversation_id}
    response_conversation = requests.post(url, json=params, headers=header)
    conversation_info = response_conversation.json()
    
    return conversation_info

In [16]:
generate_act_conv_info('754896')

{'conversation_id': '754896',
 'messages': [{'message_id': '754896-0',
   'text': "Hi! I wanted to know where my machine is. I'm going on a vacation next week and I won't be here to receive the machine, so I'd like to know what day the delivery is planned to let someone pick it up if I'm not home",
   'created_at': '1640197675'}],
 'merchant_id': '395023',
 'subject': 'check delivery address'}

In [17]:
def generate_info_of_conv(conversation_id, info_needed):
    
    conversation_id = str(conversation_id)
    info_needed = str(info_needed)
    
    url = conversations_api+HTTP_request_conversation_info
    params = {"conversation_id": conversation_id}
    response_conversation = requests.post(url, json=params, headers=header)
    conversation_info = response_conversation.json()

    info = conversation_info.get(info_needed)
    
    return info

In [18]:
generate_info_of_conv('754893', 'messages')


[{'message_id': '754893-0',
  'text': "hello, my money hasn't landed in my bank account yet. I want to know what happened",
  'created_at': '1640197668'}]

## treat one conversation by one

In [19]:
generate_act_convs_list()

[754893, 754894, 754895, 754896, 754897]

# USE this pipeline for answering the questions

In [47]:
def responses_pipeline():
    active_chats = generate_act_convs_list()
    subject_list = []
#     while len(active_chats) > 0:
    for chat in list(active_chats): #parse all the active chats
        #parse all the active messages in a conversation
        messages = generate_info_of_conv(chat, 'messages')
        for message in list(messages):

            #generate the subject of the problem and give it to the bot AI
            subject = generate_info_of_conv(chat,'subject')
            subject_list.append(subject)
            messages.remove(message)
            
        
        active_chats.remove(chat)
    return subject_list

In [48]:
responses_pipeline()

['bank account receipt problem',
 'connection problem',
 'late delivery',
 'check delivery address',
 'delivery forecast']

# Delivery Response

## Check delivery API status

In [32]:
def delivery_api_status(id_sale, zip_code):
    #urls
    sale_url= sale_id_api+HTTP_request_sale_track
    zip_url = sale_id_api+HTTP_request_sale_zip 
    
    
    #API response
    response_sale_track = requests.post(sale_url, json={'id_sale':id_sale}, headers=header).json() #HARD CODE
    response_sale_zip = requests.post(zip_url, json={'zip_code':zip_code}, headers=header).json()
    
    #API Problem checker
    api_sale = api_problem(response_sale_track)
    api_zip = api_problem(response_sale_zip)
    
    #lista y si alguno es error regresar zip error
    
    return (api_sale, api_zip)

In [33]:
delivery_api_status('123457','02219001')

(None, 'error')

In [24]:
#get id sale from SQL
id_sale = get_sale_id_SQL('849583')

#get info from tracking API
tracking_info = generate_sale_tracking_info(id_sale)
#get info zip code API
zip_code = tracking_info.get("destination_zip_code")
zip_code_info = generate_sale_zip_info({'zip_code':zip_code})
zip_code_info

{'error': 'Internal error'}

In [36]:
#try api error
id_sale = get_sale_id_SQL('849583')
print(id_sale)
tracking_info = generate_sale_tracking_info(id_sale)
print(tracking_info)
zip_code = tracking_info()

{'id_sale': '123458'}


{'id': '123458',
 'status': 'In transit between distribution centers',
 'delivery_forecast': '01/03/2022',
 'destination_zip_code': '60811340'}

In [25]:
def api_problem(dictionary):
    api_error = ['Internal error', 'NA', 'error']
    for word in api_error:
        if word in dictionary:
            return word

In [26]:
api_problem(zip_code_info)

'error'

## Response

In [27]:
id_sale = get_sale_id_SQL('395023')
id_sale

{'id_sale': '123457'}

In [28]:
generate_sale_tracking_info(id_sale)

{'id': '123457',
 'status': 'In customs clearance in Brazil',
 'delivery_forecast': '01/02/2022',
 'destination_zip_code': '02219001'}

In [29]:
merchant_id = '384923'
# 395023
# 849583
# 384923
# 558392 530364 384923
#TBD check that all api are [200] if not -> answer internar error 

#--------------------------------
#api_status = delivery_api_status(id_sale=id_sale, zip_code=zip_code)
# api_status = delivery_api_status()

# if api_status == 'err':
#     print('internal error')
# else:
#     #the rest
#--------------------------------

#get id sale from SQL
id_sale = get_sale_id_SQL(merchant_id)

#get info from tracking API
tracking_info = generate_sale_tracking_info(id_sale)

#get info zip code API
zip_code = tracking_info.get("destination_zip_code")
zip_code_info = generate_sale_zip_info({'zip_code':zip_code})
#variables
delivery_forecast = tracking_info.get('delivery_forecast')
order_status = tracking_info.get('status')
neighborhood = zip_code_info.get('neighborhood')
city = zip_code_info.get('city')
street = zip_code_info.get('street')
state = zip_code_info.get('state')
complement = zip_code_info.get('complement')

if complement is not None and isinstance(complement,str) and len(complement)>0: #do something when there is a complement
    print(f"Concerning your current issue\nYour machine it's expected to be deliver the \033[1m{delivery_forecast}\033[0m\nTo the following address:\n\033[1m{state}, {city}, {street}, {neighborhood}, {zip_code}\033[0m, \033[1m{complement}\033[0m.\nStatus: \033[1m{order_status}\033[0m")
else:
    print(f"Concerning your current issue\nYour machine it's expected to be deliver the \033[1m{delivery_forecast}\033[0m\nTo the following address:\n\033[1m{state}, {city}, {street}, {neighborhood}, {zip_code}\033[0m.\nStatus: \033[1m{order_status}\033[0m")


Concerning your current issue
Your machine it's expected to be deliver the 01/12/2021
To the following address:
MG, Belo Horizonte, Rua Professor Patrocínio Filho, Palmares, 31160550.
Status: On delivery route


In [30]:
def delivery_response(merchant_id):
    #TBD check that all api are [200] if not -> answer internar error 

    #--------------------------------
    #api_status = delivery_api_status(id_sale=id_sale, zip_code=zip_code)
    # api_status = delivery_api_status()

    # if api_status == 'err':
    #     print('internal error')
    # else:
    #     #the rest
    #--------------------------------

    #get id sale from SQL
    id_sale = get_sale_id_SQL(merchant_id)

    #get info from tracking API
    tracking_info = generate_sale_tracking_info(id_sale)

    #get info zip code API
    zip_code = tracking_info.get("destination_zip_code")
    zip_code_info = generate_sale_zip_info({'zip_code':zip_code})
    #variables
    delivery_forecast = tracking_info.get('delivery_forecast')
    order_status = tracking_info.get('status')
    neighborhood = zip_code_info.get('neighborhood')
    city = zip_code_info.get('city')
    street = zip_code_info.get('street')
    state = zip_code_info.get('state')
    complement = zip_code_info.get('complement')

    if complement is not None and isinstance(complement,str) and len(complement)>0: #do something when there is a complement
        return print(f"Concerning your current issue\nYour machine it's expected to be deliver the \033[1m{delivery_forecast}\033[0m\nTo the following address:\n\033[1m{state}, {city}, {street}, {neighborhood}, {zip_code}\033[0m, \033[1m{complement}\033[0m.\nStatus: \033[1m{order_status}\033[0m")
    else:
        return print(f"Concerning your current issue\nYour machine it's expected to be deliver the \033[1m{delivery_forecast}\033[0m\nTo the following address:\n\033[1m{state}, {city}, {street}, {neighborhood}, {zip_code}\033[0m.\nStatus: \033[1m{order_status}\033[0m")


In [31]:
delivery_response(849583)

Concerning your current issue
Your machine it's expected to be deliver the 01/03/2022
To the following address:
None, None, None, None, 60811340.
Status: In transit between distribution centers
